This notebook and subsequent analysis was adapted from SIADS 630 Causal Inference course from the Masters of Applied Data Science (MADS) program at the University of Michigan. Produced by Michael Clark.

<br>
Background: Nike claims it's Vaporfly running sneakers "are about 4% better than some of it's best racing shoes". This analysis of 24,699 runners that qualified for and ran the same marathon to determine if there is sufficient evidence to support the Company's claim.
</br>
<br>
Link to the NYT article:

[*Nike Says Its $250 Running Shoes Will Make You Run Much Faster. What if That’s Actually True?*](https://www.nytimes.com/interactive/2018/07/18/upshot/nike-vaporfly-shoe-strava.html#:~:text=Nike%20says%20the%20shoes%20are,to%20a%20four%2Dhour%20marathoner.)


In [2]:
# library imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
#pip install causalinference
from causalinference import CausalModel

# connecting Google Drive and changing dir
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/MyDrive/git/VF_causalModel')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The data file "marathon_race_data.csv” contains 5 vars for 24,699 runners that qualified for and ran the same marathon. Below are the descriptions of each variable in the data:

* age: age of runner (min value: 18, max value: 55)
* male: dummy variable for gender; equal to 0 if female, 1 if male
* marathoner_type: “seasoned” if runner has at least 3 prior completed marathons,
“enthusiastic” if runner has completed 1 or 2 prior completed marathons,
“first_timer” if this is a runner’s first time running a marathon
* vaporfly: 1 if a runner’s racing shoe is Nike Vaporfly, 0 otherwise
* race_time: marathon completion time in seconds

In [5]:
# load the CSV data into a dataframe, sample 5 records
marathon_df = pd.read_csv('marathon_race_data.csv')
marathon_df.sample(5)

,age,marathoner_type,vaporfly,race_time,male
5808,26,first_timer,1,15067.636,1
13995,34,first_timer,0,15415.063,0
3470,34,seasoned,0,12969.480,0
6062,38,first_timer,1,14265.666,0
20143,31,first_timer,0,14286.604,1


Examine the race_time variable to identify any outliers and understand its distribution properties to determine if any data manipulation or transformation is required.  

In [20]:
# convert race time to hours to make it more understandable
marathon_df['race_time_hrs'] = marathon_df.race_time / 60 / 60
marathon_df.race_time_hrs.describe(), print('fastest race time: {} hours'.format(marathon_df.race_time_hrs.min()))

fastest race time: 1.7271006944444445 hours


(count    24699.000000
 mean         3.949861
 std          0.539766
 min          1.727101
 25%          3.588125
 50%          3.958961
 75%          4.320235
 max          5.904248
 Name: race_time, dtype: float64,
 None)

In [22]:
# transform race time using natural log
marathon_df['race_time_nl'] = np.log(marathon_df.race_time)
marathon_df.race_time_nl.describe()

count    24699.000000
mean         9.552691
std          0.140757
min          8.735133
25%          9.466319
50%          9.564671
75%          9.651999
max          9.964361
Name: race_time_nl, dtype: float64

In [46]:
# analyze the means across vaporfly vs non vaporfly groups as well as the n for each group
marathon_df.groupby('vaporfly').agg({'race_time_nl':np.mean,'vaporfly':np.size})

,race_time_nl,vaporfly
vaporfly,,
0,9.585015,12228
1,9.520997,12471


Compute the means of the natural log of race time (outcome variable) for runners who ran in the Vaporfly sneakers and for those that did not. Compute the difference in means.

In [43]:
w_vf = marathon_df[marathon_df['vaporfly']==1].race_time_nl.mean()
wo_vf = marathon_df[marathon_df['vaporfly']==0].race_time_nl.mean()
diff_in_means = w_vf - wo_vf

In [44]:
diff_in_means

-0.06401793390485189

<b> ATE_1 - controlling for age </b>
<br>
Controlling for runner age, estimate ATE for wearing Nike Vaporfly shoes, using nearest neighbor matching (Euclidean distance) on the variable age using the causalinference library.


In [54]:
# create a Causal Model object with outcome variable natural log of race times (Y), treatment variable of Vaporfly (D) and control variable Age (X)
model = CausalModel(Y=marathon_df['race_time_nl'].values, D=marathon_df['vaporfly'].values, X=marathon_df['age'].values)

# estimate causal model using methods to match treated units (e.g., vaporfly wearers) with similar untreated runners based on age (X covariate)
# to make the comparison between the treatment (D) and control groups more apples-to-apples.
model.est_via_matching()

# assign results to ate_1 and print them
ate_1 = dict(model.estimates)['matching']['ate'].round(3)
ate_1

-0.041

<b> ATE_2 with Propensity for treatment </b>
<br>
Estimate the probability of a runner to wear the Vaporfly (i.e. treatment) given their characteristics (age, gender, seasoned, and enthusiastic) using a logistic regression model. Use these propensity scores to match up runners to determine the ATE of the treated.

In [53]:
# create a binary encoding for seasoned and enthusiastic runners
marathon_df['seasoned_runner'] = np.where(marathon_df['marathoner_type']=='seasoned',1,0)
marathon_df['enthusiastic_runner'] = np.where(marathon_df['marathoner_type']=='enthusiastic',1,0)

# establish covariates for the propensity matching and control
X_vars = marathon_df[['age','male','seasoned_runner','enthusiastic_runner']].values

# create model
model_2 = CausalModel(Y=marathon_df['race_time_nl'].values, D=marathon_df['vaporfly'].values, X = X_vars)

# estimate propensity and matching
model_2.est_propensity()
model_2.est_via_matching()

# assign results to ate_2 and print them
ate_2 = dict(model_2.estimates)['matching']['ate'].round(3)
ate_2

-0.043

<b> Controlled regression using robust standard errors </b>
<br>
Conduct regression analysis to measure the benefits of Vaporfly shoes on marathon race times, controlling for confounders such as age, gender, and runner experience (seasoned, enthusiastic).

In [62]:
reg_model = smf.ols(formula = 'race_time_nl ~ vaporfly + age + seasoned_runner + enthusiastic_runner',
                    data = marathon_df[['race_time_nl','vaporfly','age','seasoned_runner','enthusiastic_runner']]).fit()
reg_model.get_robustcov_results().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           race_time_nl   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     2013.
Date:                Thu, 19 Oct 2023   Prob (F-statistic):               0.00
Time:                        18:08:59   Log-Likelihood:                 17111.
No. Observations:               24699   AIC:                        -3.421e+04
Df Residuals:                   24694   BIC:                        -3.417e+04
Df Model:                           4                                         
Covariance Type:                  HC1                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               9.8786      0.004   2461.195      0.000       9.871       9.886
vaporfly               -0.0404      0.002    -25.868      0.000      -0.044      -0.037
age                    -0.0085      0.000    -71.073      0.000      -0.009      -0.008
seasoned_runner        -0.0874      0.002    -38.236      0.000      -0.092      -0.083
enthusiastic_runner    -0.0567      0.004    -15.485      0.000      -0.064      -0.050
==============================================================================
Omnibus:                      748.302   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              822.610
Skew:                          -0.428   Prob(JB):                    2.36e-179
Kurtosis:                       3.261   Cond. No.                         183.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""